## KoNLPy.Twitter

KoNLPy의 [트위터 한국어 분석기][twitterlink] ([현 오픈 한국어 분석기][openlink])는 속도도 빠르고 다양한 사전도 확보하고 있는 한국어 분석기입니다. 하지만 컴파일이 되어 있는 형태로 KoNLPy에 들어가 있기 때문에 사용자 사전의 추가가 힘들고\*, 내가 원하는 임의의 태그를 지정할 수 없습니다. 

customized_KoNLPy는 확실히 알고 있는 단어들에 대해서는 라이브러리를 거치지 않고 주어진 어절을 아는 단어들로 토크나이징 / 품사판별을 하는 기능을 제공합니다. 이를 위해 template 기반 토크나이징을 수행합니다. 

    사전: {'아이오아이': 'Noun', '는': 'Josa'}
    탬플릿: Noun + Josa
    
위와 같은 단어 리스트와 탬플릿이 있다면 '아이오아이는' 이라는 어절은 [('아이오아이', 'Noun'), ('는', 'Josa')]로 분리됩니다. 

\* Scala 코드를 이용할 경우에는 사용자 사전의 추가가 매우 쉽습니다 [참고][openlink]

[twitterlink]: https://github.com/twitter/twitter-korean-text
[openlink]: https://github.com/open-korean-text/open-korean-text

KoNLPy의 버전은 0.4.4 기준입니다. KoNLPy의 Twitter를 이용하여 '우리아이오아이는 정말 이뻐요'라는 문장을 처리하면 '아이오' + '아이'로 명사가 잘못 인식됩니다. 트와이스의 'tt' 역시 명사보다는 영어로 인식됩니다. 한국어 분석기 이지만, tt는 명사로 미리 분류하고 싶습니다. 

In [1]:
from pprint import pprint
import konlpy
konlpy.__version__

'0.5.1'

In [2]:
from konlpy.tag import Twitter as OriginalTwitter

twitter_original = OriginalTwitter()
print(twitter_original.pos('우리아이오아이는 정말 이뻐요'),'\n')
print(twitter_original.pos('트둥이꺼 tt도 좋아요'))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('우리', 'Noun'), ('아이오', 'Noun'), ('아이', 'Noun'), ('는', 'Josa'), ('정말', 'Noun'), ('이뻐요', 'Adjective')] 

[('트', 'Noun'), ('둥이', 'Noun'), ('꺼', 'Suffix'), ('tt', 'Alpha'), ('도', 'Noun'), ('좋아요', 'Adjective')]


customized_KoNLPy 에는 현재 트위터 한국어 분석기 만을 이용하는 wrapping class만 제공되고 있습니다. customized_KoNLPy의 Twitter는 본래 KoNLPy의 tag에 추가되는 함수가 있습니다. 

Twitter.add_dictionary(words, tag)는 사용자가 사전을 추가할 수 있는 부분입니다. 단어를 하나씩 추가할 수 있습니다. 추가한 뒤 Twitter의 숨김 변수인 \_dictionary.\_pos2words를 확인해보면 입력한 단어들을 볼 수 있습니다. 

## ckonlpy.Twitter

### Add word to custom-dictionary

git clone을 한 상태에서 tutorial code를 이용하신다면 아래의 코드를 실행하여 path를 추가하십시요

In [3]:
import sys
sys.path.insert(0, '../')

import ckonlpy
print('ckonlpy version = {}'.format(ckonlpy.__version__))

ckonlpy version = 0.0.63


In [4]:
from ckonlpy.tag import Twitter

# use_twitter_dictionary 의 default 는 True 입니다.
twitter = Twitter(use_twitter_dictionary=False)

twitter.add_dictionary('이', 'Modifier')
twitter.add_dictionary('우리', 'Modifier')
twitter.add_dictionary('이번', 'Modifier')
twitter.add_dictionary('아이오아이', 'Noun')
twitter.add_dictionary('행사', 'Noun')
twitter.add_dictionary('아이', 'Noun')
twitter.add_dictionary('번것', 'Noun')
twitter.add_dictionary('것', 'Noun')
twitter.add_dictionary('은', 'Josa')
twitter.add_dictionary('는', 'Josa')

if twitter.use_twitter_dictionary:
    pprint(twitter.dictionary._pos2words.keys())
else:
    pprint(twitter.dictionary._pos2words)

{'Josa': {'는', '은'},
 'Modifier': {'우리', '이', '이번'},
 'Noun': {'것', '아이오아이', '행사', '아이', '번것'}}


/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


사전을 추가한 뒤, '아이오아이'가 명사로 제대로 인식됨을 확인할 수 있습니다. 

In [5]:
twitter.pos('우리아이오아이는 정말 이뻐요')

[('우리', 'Modifier'),
 ('아이오아이', 'Noun'),
 ('는', 'Josa'),
 ('정말', 'Noun'),
 ('이뻐요', 'Adjective')]

In [6]:
twitter.pos('아이오아이 이뻐요')

[('아이오아이', 'Noun'), ('이뻐요', 'Adjective')]

사전을 추가할 때, 하나의 품사에 대하여 동시에 여러 개의 단어셋을 입력할 수도 있습니다.

Twitter.add_dictionary(words, tag)는 한번에 list of str 형식의 여러 개의 단어들을 입력할 수도 있습니다. 

In [7]:
twitter.add_dictionary(['트와이스', 'tt', '트둥이', '꺼', '우리'], 'Noun')
pprint(twitter.dictionary._pos2words)

{'Josa': {'는', '은'},
 'Modifier': {'우리', '이', '이번'},
 'Noun': {'꺼', 'tt', '행사', '번것', '트둥이', '것', '아이오아이', '트와이스', '아이', '우리'}}


In [8]:
twitter.pos('트와이스tt는 좋아요')

[('트와이스', 'Noun'), ('tt', 'Noun'), ('는', 'Josa'), ('좋아요', 'Adjective')]

### Use KoNLPy.Twitter dictionary

트위터 분석기의 조사사전을 이용할 수도 있습니다. Twitter()를 만들 때 argument를 넣을 수 있습니다. 

In [9]:
twitter1 = Twitter(use_twitter_dictionary=True)
len(twitter1.dictionary._pos2words['Josa'])

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


512

### Template
현재는 customized_tagger로 탬플릿 기반 토크나이저를 이용하고 있습니다. 어떤 탬플릿이 들어있는지 확인하기 위해서는 아래 부분을 확인하면 됩니다. 

    twitter._customized_tagger.templates
    
현재는 다음의 탬플릿이 입력되어 있습니다. 

In [10]:
twitter.template_tagger.templates

[('Adjective',),
 ('Adverb',),
 ('Conjunction',),
 ('Exclamation',),
 ('KoreanParticle',),
 ('Noun',),
 ('Verb',),
 ('Noun', 'Noun'),
 ('Noun', 'Josa'),
 ('Noun', 'Adjective'),
 ('Noun', 'Verb'),
 ('Modifier', 'Noun'),
 ('Noun', 'Noun', 'Adjective'),
 ('Noun', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Verb'),
 ('Modifier', 'Noun', 'Josa')]

기본 탬플릿은 customized_konlpy/data/templates/twitter_templates0 에 저장되어 있습니다. text 형식의 파일이며, 띄어쓰기로 아래와 같은 기본 템플릿을 지정하면 됩니다. 

In [11]:
cat ../ckonlpy/data/templates/twitter_templates0

Adjective
Adverb
Conjunction
Exclamation
KoreanParticle
Noun
Verb
Noun Noun
Noun Josa
Noun Adjective
Noun Verb
Modifier Noun
Noun Noun Adjective
Noun Noun Josa
Noun Noun Verb
Modifier Noun Josa

('Noun', 'Noun', 'Josa')가 입력되었고, '트와이스', 'tt'가 명사인지 알고 있기 때문에 아래 문장은 제대로 인식이 됩니다. 

In [12]:
twitter.pos('우리트둥이tt는 좋아요')

[('우리', 'Noun'),
 ('트둥이', 'Noun'),
 ('tt', 'Noun'),
 ('는', 'Josa'),
 ('좋아요', 'Adjective')]

(Noun, Noun, Noun, Josa) 가 template 에 없지만, (Noun, ) 과 (Noun, Noun, Josa) 가 있기 때문에 매칭이 됩니다. 

In [13]:
twitter.pos('우리트둥이tt는 좋아요')

[('우리', 'Noun'),
 ('트둥이', 'Noun'),
 ('tt', 'Noun'),
 ('는', 'Josa'),
 ('좋아요', 'Adjective')]

버전 0.0.5x 까지는 어절 전체가 template 에 매칭될 때에만 template tagger 를 이용하였습니다. 하지만, 0.0.6+ 이상에서는 어절의 중간에 template matching 이 되어도 template tagger 를 이용합니다.

이전처럼 어절 전체가 template 으로 완전히 매칭될 때만 template tagger 를 이용하고 싶다면 perfect_match = True 로 설정합니다.

(Noun, Noun, Noun, Josa) template 은 없기 때문에 perfect_match = True 로 설정하면, 어절을 base tagger 를 이용하여 품사 판별을 수행합니다.

In [14]:
twitter.pos('우리트둥이tt는 좋아요', perfect_match=True)

[('우', 'Adverb'),
 ('리트', 'Noun'),
 ('둥이', 'Noun'),
 ('tt', 'Alpha'),
 ('는', 'Verb'),
 ('좋아요', 'Adjective')]

작업 중 탬플릿을 추가하고 싶다면, 탬플릿은 하나 단위로 tuple of str의 형식으로 입력할 수 있습니다.

\_customized_tagger.add_a_templated()은 중복되는 탬플릿이 아닌지 확인한 다음 탬플릿을 추가하는 함수입니다. 




In [15]:
twitter.template_tagger.add_a_template(('Noun', 'Noun', 'Noun', 'Josa'))
twitter.template_tagger.templates

[('Adjective',),
 ('Adverb',),
 ('Conjunction',),
 ('Exclamation',),
 ('KoreanParticle',),
 ('Noun',),
 ('Verb',),
 ('Noun', 'Noun'),
 ('Noun', 'Josa'),
 ('Noun', 'Adjective'),
 ('Noun', 'Verb'),
 ('Modifier', 'Noun'),
 ('Noun', 'Noun', 'Adjective'),
 ('Noun', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Verb'),
 ('Modifier', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Noun', 'Josa')]

Template 을 추가하면 perfect_match 일 때에도 매칭이 되는 template 이 있기 때문에 동일한 결과가 출력됩니다.

In [16]:
twitter.pos('우리트둥이tt는 좋아요', perfect_match=True)

[('우리', 'Noun'),
 ('트둥이', 'Noun'),
 ('tt', 'Noun'),
 ('는', 'Josa'),
 ('좋아요', 'Adjective')]

### Out of tags

사전을 추가할 때, 트위터 한국어 분석기에 존재하지 않는 태그가 들어가는 것을 방지하기 위해 tag의 값을 확인하는 부분이 구현되어 있습니다. 

    twitter.tagset
    
    >>> {'Adjective': '형용사',
         'Adverb': '부사',
         'Alpha': '알파벳',
         'Conjunction': '접속사',
         'Determiner': '관형사',
         'Eomi': '어미',
         'Exclamation': '감탄사',
         'Foreign': '외국어, 한자 및 기타기호',
         'Hashtag': '트위터 해쉬태그',
         'Josa': '조사',
         'KoreanParticle': '(ex: ㅋㅋ)',
         'Modifier': '관형사',
         'Noun': '명사',
         'Number': '숫자',
         'PreEomi': '선어말어미',
         'Punctuation': '구두점',
         'ScreenName': '트위터 아이디',
         'Suffix': '접미사',
         'Unknown': '미등록어',
         'Verb': '동사'}
         
twitter.tagset에 등록되어 있지 않는 품사에 대해서는 ValueError를 raise 합니다. 

In [17]:
twitter.add_dictionary('lovit', 'Name')

ValueError: Name is not available tag

하지만 Twitter.add_dictionary(words, tag, force=True)로 단어를 사전에 입력하면 알려지지 않은 품사라 하더라도 입력할 수 있습니다. 

In [18]:
twitter.add_dictionary('lovit', 'Name', force=True)
print(twitter.dictionary._pos2words.keys())

dict_keys(['Modifier', 'Noun', 'Josa', 'Name'])


알려지지 않은 품사를 단어 사전에 추가하면 template 에 해당 단어가 잡힐 수 있도록 자동으로 (tag,) 라는 템플릿이 추가됩니다.

In [19]:
twitter.template_tagger.templates

[('Adjective',),
 ('Adverb',),
 ('Conjunction',),
 ('Exclamation',),
 ('KoreanParticle',),
 ('Noun',),
 ('Verb',),
 ('Noun', 'Noun'),
 ('Noun', 'Josa'),
 ('Noun', 'Adjective'),
 ('Noun', 'Verb'),
 ('Modifier', 'Noun'),
 ('Noun', 'Noun', 'Adjective'),
 ('Noun', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Verb'),
 ('Modifier', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Noun', 'Josa'),
 ('Name',)]

하지만 아래 문장에서 ('Name', 'Josa') 라는 template 이 없기 때문에 '은'이 명사로 처리됩니다.

In [20]:
twitter.pos('lovit은 이름입니다')

[('lovit', 'Name'), ('은', 'Noun'), ('이름', 'Noun'), ('입니다', 'Adjective')]

'Name'이라는 클래스 (더이상 품사가 아니므로)를 이용하는 탬플릿을 하나 입력한 뒤 pos에 입력하면 어절 'lovit은' customized_tagger에 의하여 처리가 되고, 사용자 사전에 알려지지 않은 어절은 본래의 트위터 분석기에 의하여 처리가 됩니다. 

In [21]:
twitter.template_tagger.add_a_template(('Name', 'Josa'))
pprint(twitter.template_tagger.templates)
twitter.pos('lovit은 이름입니다.')

[('Adjective',),
 ('Adverb',),
 ('Conjunction',),
 ('Exclamation',),
 ('KoreanParticle',),
 ('Noun',),
 ('Verb',),
 ('Noun', 'Noun'),
 ('Noun', 'Josa'),
 ('Noun', 'Adjective'),
 ('Noun', 'Verb'),
 ('Modifier', 'Noun'),
 ('Noun', 'Noun', 'Adjective'),
 ('Noun', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Verb'),
 ('Modifier', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Noun', 'Josa'),
 ('Name',),
 ('Name', 'Josa')]


[('lovit', 'Name'),
 ('은', 'Josa'),
 ('이름', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation')]

### Scoring

Templates를 이용하여도 후보가 여러 개 나올 수 있습니다. 여러 개 후보 중에서 best 를 선택하는 함수를 직접 디자인 할 수 도 있습니다. 이처럼 몇 개의 점수 기준을 만들고, 각 기준의 weight를 부여하는 방식은 트위터 분석기에서 이용하는 방식인데, 직관적이고 튜닝 가능해서 매우 좋은 방식이라 생각합니다. 

In [22]:
my_weights = [
    ('num_nouns', -0.1), 
    ('num_words', -0.2),
    ('no_noun', -1),
    ('len_sum_of_nouns', 0.2)
]

def my_evaluate_function(candidate):
    num_nouns = len([word for word, pos, begin, e in candidate if pos == 'Noun'])
    num_words = len(candidate)
    has_no_nouns = (num_nouns == 0)
    len_sum_of_nouns = 0 if has_no_nouns else sum(
        (len(word) for word, pos, _, _ in candidate if pos == 'Noun'))

    scores = (num_nouns, num_words, has_no_nouns, len_sum_of_nouns)
    score = sum((score * weight for score, (_, weight) in zip(scores, my_weights)))
    return score

twitter.set_evaluator(my_weights, my_evaluate_function, test=True)

candidate
[('이', 'Noun', 0, 1),
 ('것', 'Noun', 1, 2),
 ('은', 'Josa', 2, 3),
 ('테', 'Noun', 3, 4),
 ('스트', 'Noun', 4, 6)]
score = -0.3999999999999999

candidate
[('이것', 'Noun', 0, 2),
 ('은', 'Josa', 2, 3),
 ('테', 'Noun', 3, 4),
 ('스트', 'Noun', 4, 6)]
score = -0.10000000000000009

candidate
[('이것', 'Noun', 0, 2), ('은', 'Josa', 2, 3), ('테스트', 'Noun', 3, 6)]
score = 0.19999999999999996



### Stem, Norm

pos 함수에서 norm, stem 옵션을 사용할 수 있습니다.

In [23]:
twitter.pos('테스트입니다', norm=True, stem=True)

[('테스트', 'Noun'), ('이다', 'Adjective')]

In [24]:
twitter.pos('테스트입니다', norm=True, stem=False)

[('테스트', 'Noun'), ('입니다', 'Adjective')]

In [25]:
twitter.pos('테스트입니다', norm=False, stem=True)

[('테스트', 'Noun'), ('이다', 'Adjective')]

### Postprocessing

passwords, stopwords, 특정 품사, 혹은 미리 정의된 사전을 이용하여 단어를 변환하여 출력합니다. 

passwords 는 단어, (단어, 품사)를 입력하면 해당 단어 외에는 출력하지 않습니다. 

In [26]:
from ckonlpy.tag import Postprocessor

passwords = {'아이오아이', ('정말', 'Noun')}
postprocessor = Postprocessor(
    base_tagger = twitter,
    passwords = passwords
)

sent = '우리아이오아이는 정말 이뻐요'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('는', 'Verb'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]

after  : [('아이오아이', 'Noun'), ('정말', 'Noun')]


stopwords 에 등록된 단어, (단어, 품사)는 pos 결과에서 제외됩니다. 

In [27]:
stopwords = {'는'}

postprocessor = Postprocessor(
    base_tagger = twitter,
    stopwords = stopwords
)

sent = '우리아이오아이는 정말 이뻐요'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('는', 'Verb'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]

after  : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]


특정 품사를 지정하면 해당 품사만 pos 에서 출력됩니다. 

In [28]:
passtags = {'Noun'}

postprocessor = Postprocessor(
    base_tagger = twitter,
    passtags = passtags
)

sent = '우리아이오아이는 정말 이뻐요'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('는', 'Verb'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]

after  : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('정말', 'Noun')]


치환할 단어, (단어, 품사)를 dict 형식으로 정의하면 pos 에서 단어가 치환되어 출력됩니다. 

In [29]:
replace = {
    '아이오아이': '아이돌',
    ('이뻐', 'Adjective'): '예쁘다'
}

postprocessor = Postprocessor(
    base_tagger = twitter,
    replace = replace
)

sent = '우리아이오아이는 정말 이뻐요'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('우리', 'Noun'), ('아이오아이', 'Noun'), ('는', 'Verb'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]

after  : [('우리', 'Noun'), ('아이돌', 'Noun'), ('는', 'Verb'), ('정말', 'Noun'), ('이뻐요', 'Adjective')]


하나의 단어로 묶을 단어열을 tuple of str 로 만든 뒤, 품사와 함께 nested tuple 로 만듭니다. ngram 에 입력된 단어열은 ngram pos tag 로 합쳐져 출력됩니다. ngram pos tag 는 임의의 태그 이름을 넣을 수 있습니다.

In [30]:
ngrams = [(('미스', '함무라비'), 'Noun'), (('바람', '의', '나라'), 'Game')]

postprocessor = Postprocessor(
    base_tagger = twitter,
    ngrams = ngrams
)

sent = '미스 함무라비는 재밌는 드라마입니다'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s\n\n' % postprocessor.pos(sent))

sent = '바람의 나라는 게임이름 입니다'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('미스', 'Noun'), ('함무라비', 'Noun'), ('는', 'Josa'), ('재밌는', 'Adjective'), ('드라마', 'Noun'), ('입니다', 'Adjective')]

after  : [('미스 - 함무라비', 'Noun'), ('는', 'Josa'), ('재밌는', 'Adjective'), ('드라마', 'Noun'), ('입니다', 'Adjective')]


before : [('바람', 'Noun'), ('의', 'Josa'), ('나라', 'Noun'), ('는', 'Josa'), ('게임', 'Noun'), ('이름', 'Noun'), ('입니다', 'Adjective')]

after  : [('바람 - 의 - 나라', 'Game'), ('는', 'Josa'), ('게임', 'Noun'), ('이름', 'Noun'), ('입니다', 'Adjective')]


단어열만 입력한 경우에는 ngram 의 품사를 명사로 가정합니다.

In [31]:
ngrams = [('미스', '함무라비'), ('바람', '의', '나라')]

postprocessor = Postprocessor(
    base_tagger = twitter,
    ngrams = ngrams
)

sent = '미스 함무라비는 재밌는 드라마입니다'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s\n\n' % postprocessor.pos(sent))

sent = '바람의 나라는 게임이름 입니다'
print('before : %s\n' % twitter.pos(sent))
print('after  : %s' % postprocessor.pos(sent))

before : [('미스', 'Noun'), ('함무라비', 'Noun'), ('는', 'Josa'), ('재밌는', 'Adjective'), ('드라마', 'Noun'), ('입니다', 'Adjective')]

after  : [('미스 - 함무라비', 'Noun'), ('는', 'Josa'), ('재밌는', 'Adjective'), ('드라마', 'Noun'), ('입니다', 'Adjective')]


before : [('바람', 'Noun'), ('의', 'Josa'), ('나라', 'Noun'), ('는', 'Josa'), ('게임', 'Noun'), ('이름', 'Noun'), ('입니다', 'Adjective')]

after  : [('바람 - 의 - 나라', 'Noun'), ('는', 'Josa'), ('게임', 'Noun'), ('이름', 'Noun'), ('입니다', 'Adjective')]


## Loading wordset

utils 에는 stopwords, passwords, replace word pair 를 파일로 저장하였을 경우, 이를 손쉽게 불러오는 함수가 있습니다.

load_wordset 은 set of str 혹은 set of tuple 을 return 합니다. 예시의 passwords.txt 의 내용은 아래와 같습니다. 단어의 품사는 한 칸 띄어쓰기로 구분합니다.

    아이오아이
    아이오아이 Noun
    공연

In [32]:
from ckonlpy.utils import load_wordset

passwords = load_wordset('./passwords.txt')
print(passwords)

{'공연', '아이오아이', ('아이오아이', 'Noun')}


stopwords.txt 의 포멧은 passwords.txt 와 같습니다. 단어의 품사가 있을 경우 한 칸 띄어쓰기로 구분합니다.

    은
    는
    이 Josa

In [33]:
stopwords = load_wordset('./stopwords.txt')
print(stopwords)

{('이', 'Josa'), '는', '은'}


치환할 단어쌍은 tap 구분이 되어있습니다. 치환될 단어에 품사 태그가 있을 경우 한 칸 띄어쓰기로 구분합니다.

    str\tstr
    str str\tstr

아래는 replacewords.txt 의 예시입니다.

    아빠	아버지
    엄마 Noun	어머니

In [34]:
from ckonlpy.utils import load_replace_wordpair

replace = load_replace_wordpair('./replacewords.txt')
print(replace)

{'아빠': '아버지', ('엄마', 'Noun'): '어머니'}


ngram 은 띄어쓰기 단위로 ngram 의 구성 단어를 기술합니다. tap 기준으로 ngram pos tag 를 저장합니다. pos tag 가 없다면 단어열만 포함합니다.

    str str
    str str str\tstr

In [35]:
from ckonlpy.utils import load_ngram

ngrams = load_ngram('./ngrams.txt')
pprint(ngrams)

[('바람', '의', '나라'), (('미스', '함무라비'), 'Noun')]
